In [20]:
!pip install contractions
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
import pandas as pd
import unicodedata
import re
import contractions
import string

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from nltk.corpus import stopwords

import pyLDAvis
import pyLDAvis.gensim_models

Prepocessing the tweet data corpus

In [22]:
df = pd.read_parquet('/content/final_twitter_data.parquet')
df.head()

,timestamp,tweet_id,conversation_id,author_id,text,retweet_count,reply_count,like_count,quote_count,referenced_tweets,hashtags,tweet_type
0,2021-06-20T21:33:30.000Z,1406726930888540160,1406726930888540160,2981738470,RT @davidlohner: #IchbinHanna und schreibe auc...,7,0,0,0,"[{'type': 'retweeted', 'id': '1406714909258362...",['IchbinHanna'],retweet
1,2021-06-20T21:29:24.000Z,1406725899744157698,1406725899744157698,2981738470,Wenn Wirtschaftsjounalist:innen über #IchbinHa...,9,1,62,4,[],['IchbinHanna'],original
2,2021-06-20T20:50:42.000Z,1406716159064485895,1406716159064485895,2981738470,RT @DrKEichhorn: Wieder ein Artikel ohne echte...,16,0,0,0,"[{'type': 'retweeted', 'id': '1406697315159752...",[],retweet
3,2021-06-20T17:56:03.000Z,1406672208056012807,1406672208056012807,2981738470,RT @chrismeyer2203: So sieht es aus ⬇️ oder an...,7,0,0,0,"[{'type': 'retweeted', 'id': '1406175728190865...",[],retweet
4,2021-06-20T17:53:00.000Z,1406671439328055301,1406671439328055301,2981738470,RT @steffen_siegel: Modelle einer Universitäts...,17,0,0,0,"[{'type': 'retweeted', 'id': '1406666454368493...",[],retweet


Removing tweet_type == 'retweet' as it is not providing any new text information

In [23]:
new_df = df.loc[df['tweet_type'] != 'retweet']
new_df.head()

,timestamp,tweet_id,conversation_id,author_id,text,retweet_count,reply_count,like_count,quote_count,referenced_tweets,hashtags,tweet_type
1,2021-06-20T21:29:24.000Z,1406725899744157698,1406725899744157698,2981738470,Wenn Wirtschaftsjounalist:innen über #IchbinHa...,9,1,62,4,[],['IchbinHanna'],original
5,2021-06-20T16:17:25.000Z,1406647386542325764,1406647386542325764,2981738470,☝️⬇️ #IchbinHanna #PeerReview https://t.co/PFN...,0,0,1,0,"[{'type': 'quoted', 'id': '1406620276822061057'}]","['IchbinHanna', 'PeerReview']",original
11,2021-06-18T13:10:36.000Z,1405875593711964166,1405875593711964166,1132055796571877376,Thread ⬇️ about the cruel #precarity in German...,5,0,19,0,"[{'type': 'quoted', 'id': '1405846267759054851'}]","['precarity', 'academia', 'IchbinHanna', 'Acad...",original
16,2021-06-18T10:21:31.000Z,1405833045224087555,1405833045224087555,242424959,"Liebe @unidue und Forsa - schön, dass ihr euch...",0,1,2,0,[],"['Mittelbau', 'IchBinHanna']",original
19,2021-06-18T08:35:29.000Z,1405806358335832065,1405806358335832065,1132055796571877376,Wichtiger Thread ⬇️ zu #Machtmissbrauch in der...,2,0,13,0,"[{'type': 'quoted', 'id': '1405494574533984264'}]","['Machtmissbrauch', 'Wissenschaft', 'prekär', ...",original


In [24]:
def to_lowercase(text):
    return text.lower()

new_df.text = new_df.text.apply(to_lowercase)

new_df.head()

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,timestamp,tweet_id,conversation_id,author_id,text,retweet_count,reply_count,like_count,quote_count,referenced_tweets,hashtags,tweet_type
1,2021-06-20T21:29:24.000Z,1406725899744157698,1406725899744157698,2981738470,wenn wirtschaftsjounalist:innen über #ichbinha...,9,1,62,4,[],['IchbinHanna'],original
5,2021-06-20T16:17:25.000Z,1406647386542325764,1406647386542325764,2981738470,☝️⬇️ #ichbinhanna #peerreview https://t.co/pfn...,0,0,1,0,"[{'type': 'quoted', 'id': '1406620276822061057'}]","['IchbinHanna', 'PeerReview']",original
11,2021-06-18T13:10:36.000Z,1405875593711964166,1405875593711964166,1132055796571877376,thread ⬇️ about the cruel #precarity in german...,5,0,19,0,"[{'type': 'quoted', 'id': '1405846267759054851'}]","['precarity', 'academia', 'IchbinHanna', 'Acad...",original
16,2021-06-18T10:21:31.000Z,1405833045224087555,1405833045224087555,242424959,"liebe @unidue und forsa - schön, dass ihr euch...",0,1,2,0,[],"['Mittelbau', 'IchBinHanna']",original
19,2021-06-18T08:35:29.000Z,1405806358335832065,1405806358335832065,1132055796571877376,wichtiger thread ⬇️ zu #machtmissbrauch in der...,2,0,13,0,"[{'type': 'quoted', 'id': '1405494574533984264'}]","['Machtmissbrauch', 'Wissenschaft', 'prekär', ...",original


In [25]:
def remove_mentions_and_tags(text): #Removing mentions and hashtags
    text = re.sub(r'@\S*', '', text)
    return re.sub(r'#\S*', '', text)

new_df.text = new_df.text.apply(remove_mentions_and_tags)

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [26]:
#Removing german stop words
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

german_stop_words = stopwords.words('german')

def stop_word_removal(x):
    token = x.split()
    return ' '.join([w for w in token if not w in german_stop_words])

new_df.text = new_df.text.apply(stop_word_removal)
new_df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,timestamp,tweet_id,conversation_id,author_id,text,retweet_count,reply_count,like_count,quote_count,referenced_tweets,hashtags,tweet_type
1,2021-06-20T21:29:24.000Z,1406725899744157698,1406725899744157698,2981738470,"wirtschaftsjounalist:innen schreiben &amp; ""di...",9,1,62,4,[],['IchbinHanna'],original
5,2021-06-20T16:17:25.000Z,1406647386542325764,1406647386542325764,2981738470,☝️⬇️ https://t.co/pfnroveoas,0,0,1,0,"[{'type': 'quoted', 'id': '1406620276822061057'}]","['IchbinHanna', 'PeerReview']",original
11,2021-06-18T13:10:36.000Z,1405875593711964166,1405875593711964166,1132055796571877376,"thread ⬇️ about the cruel german the debate, w...",5,0,19,0,"[{'type': 'quoted', 'id': '1405846267759054851'}]","['precarity', 'academia', 'IchbinHanna', 'Acad...",original
16,2021-06-18T10:21:31.000Z,1405833045224087555,1405833045224087555,242424959,"liebe forsa - schön, per umfrage interessiert....",0,1,2,0,[],"['Mittelbau', 'IchBinHanna']",original
19,2021-06-18T08:35:29.000Z,1405806358335832065,1405806358335832065,1132055796571877376,wichtiger thread ⬇️ arbeitsverträge begünstigt...,2,0,13,0,"[{'type': 'quoted', 'id': '1405494574533984264'}]","['Machtmissbrauch', 'Wissenschaft', 'prekär', ...",original


In [27]:
#remove emojis

import re
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

new_df.text = new_df.text.apply(remove_emojis)
new_df.head()

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,timestamp,tweet_id,conversation_id,author_id,text,retweet_count,reply_count,like_count,quote_count,referenced_tweets,hashtags,tweet_type
1,2021-06-20T21:29:24.000Z,1406725899744157698,1406725899744157698,2981738470,"wirtschaftsjounalist:innen schreiben &amp; ""di...",9,1,62,4,[],['IchbinHanna'],original
5,2021-06-20T16:17:25.000Z,1406647386542325764,1406647386542325764,2981738470,https://t.co/pfnroveoas,0,0,1,0,"[{'type': 'quoted', 'id': '1406620276822061057'}]","['IchbinHanna', 'PeerReview']",original
11,2021-06-18T13:10:36.000Z,1405875593711964166,1405875593711964166,1132055796571877376,"thread about the cruel german the debate, whi...",5,0,19,0,"[{'type': 'quoted', 'id': '1405846267759054851'}]","['precarity', 'academia', 'IchbinHanna', 'Acad...",original
16,2021-06-18T10:21:31.000Z,1405833045224087555,1405833045224087555,242424959,"liebe forsa - schön, per umfrage interessiert....",0,1,2,0,[],"['Mittelbau', 'IchBinHanna']",original
19,2021-06-18T08:35:29.000Z,1405806358335832065,1405806358335832065,1132055796571877376,wichtiger thread arbeitsverträge begünstigt w...,2,0,13,0,"[{'type': 'quoted', 'id': '1405494574533984264'}]","['Machtmissbrauch', 'Wissenschaft', 'prekär', ...",original


LDA for base-line comparison with other algorithms

In [28]:
def generate_tokens(tweet):
  words=[]
  for word in tweet.split(' '):
  # using the if condition because we introduced extra spaces during text cleaning
    if word!='':
      words.append(word)
  return words
#storing the generated tokens in a new column named 'words'
new_df['tokens'] = new_df.text.apply(generate_tokens)
new_df.head()

<ipython-input-28-2aa5b857094b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tokens'] = new_df.text.apply(generate_tokens)


,timestamp,tweet_id,conversation_id,author_id,text,retweet_count,reply_count,like_count,quote_count,referenced_tweets,hashtags,tweet_type,tokens
1,2021-06-20T21:29:24.000Z,1406725899744157698,1406725899744157698,2981738470,"wirtschaftsjounalist:innen schreiben &amp; ""di...",9,1,62,4,[],['IchbinHanna'],original,"[wirtschaftsjounalist:innen, schreiben, &amp;,..."
5,2021-06-20T16:17:25.000Z,1406647386542325764,1406647386542325764,2981738470,https://t.co/pfnroveoas,0,0,1,0,"[{'type': 'quoted', 'id': '1406620276822061057'}]","['IchbinHanna', 'PeerReview']",original,[https://t.co/pfnroveoas]
11,2021-06-18T13:10:36.000Z,1405875593711964166,1405875593711964166,1132055796571877376,"thread about the cruel german the debate, whi...",5,0,19,0,"[{'type': 'quoted', 'id': '1405846267759054851'}]","['precarity', 'academia', 'IchbinHanna', 'Acad...",original,"[thread, about, the, cruel, german, the, debat..."
16,2021-06-18T10:21:31.000Z,1405833045224087555,1405833045224087555,242424959,"liebe forsa - schön, per umfrage interessiert....",0,1,2,0,[],"['Mittelbau', 'IchBinHanna']",original,"[liebe, forsa, -, schön,, per, umfrage, intere..."
19,2021-06-18T08:35:29.000Z,1405806358335832065,1405806358335832065,1132055796571877376,wichtiger thread arbeitsverträge begünstigt w...,2,0,13,0,"[{'type': 'quoted', 'id': '1405494574533984264'}]","['Machtmissbrauch', 'Wissenschaft', 'prekär', ...",original,"[wichtiger, thread, arbeitsverträge, begünstig..."


In [29]:
def create_dictionary(words):
  return corpora.Dictionary(words)
#passing the dataframe column having tokens as the argument
id2word = create_dictionary(new_df.tokens)
print(id2word)

Dictionary(157403 unique tokens: ['"die', '&amp;', '(ht', ')', '1/5']...)


In [30]:
def create_document_matrix(tokens, id2word):
  corpus = []
  for text in tokens:
    corpus.append(id2word.doc2bow(text))
  return corpus
#passing the dataframe column having tokens and dictionary
corpus = create_document_matrix(new_df.tokens, id2word)
# print(new_df.tokens[0])
print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1)]


In [31]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=10, random_state=100)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

In [32]:
def get_lda_topics(model, num_topics, top_n_words):
     word_dict = {}
     for i in range(num_topics):
         word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in model.show_topic(i, topn = top_n_words)];
 
     return pd.DataFrame(word_dict)
get_lda_topics(lda_model,10,10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,mehr,&amp;,&amp;,the,&amp;,of,-,mehr,-,&amp;
1,stellen,heute,viele,to,-,the,heute,&amp;,hochschulen,uhr
2,promotion,stunde,wissenschaft,and,hanna.,i,&amp;,mal,mehr,-
3,unbefristete,de,arbeitsbedingungen,a,hochschulen,a,berliner,schon,jahren,"professor,"
4,100%,wissenschaft.,1,of,wissenschaft,to,thema,ja,gut,heute
5,aufgrund,wissenschaft,arbeit,is,–,&amp;,mal,-,arbeiten,(
6,jahre,berlin,evaluation,for,erklärung,my,promotionen,immer,mal,forschung
7,dauerstellen,hanna,bin.,are,wissenschaftler*innen,s,wissenschaft,einfach,gar,lehre
8,-,wisszeitvg,forschung,german,deutschen,have,antrag,wer,wurde,wissenschaft
9,sollten,geht,lehre,about,system,for,neue,uni,viele,debatte


In [33]:
#visualising LDA
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds='mmds', R=30)
vis

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.033693  0.061487       1        1  13.893713
3      0.262041  0.387123       2        1  11.936624
2     -0.172518 -0.154024       3        1  11.033214
8     -0.160191  0.332866       4        1  10.998519
0      0.118297 -0.309121       5        1   9.929321
6     -0.266913  0.158908       6        1   9.526440
4     -0.136455 -0.383749       7        1   8.679862
9     -0.413567 -0.065362       8        1   8.311414
5      0.402989  0.169609       9        1   8.190947
1      0.332624 -0.197738      10        1   7.499946, topic_info=                    Term          Freq         Total Category  logprob  \
29                   the  22313.000000  22313.000000  Default  30.0000   
754                   to  15457.000000  15457.000000  Default  29.0000   
383                   of  14382.000000  14382.000000  Default  28.0000   
370                  and  12224.000000  12224.000000  Default  27.0000   
367                    a  12221.000000  12221.000000  Default  26.0000   
...                  ...           ...           ...      ...      ...   
43                  geht    867.344162   6358.772515  Topic10  -5.5602   
17    wissenschaftlichen    654.347376   2027.948827  Topic10  -5.8420   
4069                 via    594.086821   1292.002823  Topic10  -5.9386   
586                  uni    722.405672   4816.916062  Topic10  -5.7430   
66                   wer    633.937576   4978.099629  Topic10  -5.8736   

      loglift  
29    30.0000  
754   29.0000  
383   28.0000  
370   27.0000  
367   26.0000  
...       ...  
43     0.5981  
17     1.4591  
4069   1.8134  
586    0.6930  
66     0.5294  

[646 rows x 6 columns], token_table=       Topic      Freq                Term
term                                      
10700      5  0.997896                   !
3803       6  0.883948                   "
3803      10  0.115239                   "
47926      8  0.998111  "außenwahrnehmung"
47927      8  0.999227       "doktorvater"
...      ...       ...                 ...
46586     10  0.997475     „forscherinnen“
46587     10  0.999104          „praktisch
6988       5  0.139795                   …
6988       7  0.859927                   …
4061      10  0.999348                   ⁦

[1240 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 4, 3, 9, 1, 7, 5, 10, 6, 2])

Biterm Topic Modelling (BTM) Algorithm 
Ref: https://www.cs.toronto.edu/~jstolee/projects/topic.pdf

In [43]:
!pip install bitermplus
!pip install tmplot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.3 MB/s eta 0:00:00
  Created wheel for tmplot: filename=tmplot-0.0.10-py3-none-any.whl size=17359 sha256=6eb407c7780c6a65dad905797164d5172b9c38f45b8993c902367a7db97ca213
  Stored in directory: /root/.cache/pip/wheels/b4/ab/d1/961488a71185aae7f0266bae71068f916d4710fa9a5cbff103
Successfully built tmplot


In [52]:
import bitermplus as btm

tweet_text_btm = new_df['text'].str.strip().tolist()

X, vocabulary, vocab_dict = btm.get_words_freqs(tweet_text_btm)
tf = np.array(X.sum(axis=0)).ravel()

docs_vec = btm.get_vectorized_docs(tweet_text_btm, vocabulary)
docs_lens = list(map(len, docs_vec))

biterms = btm.get_biterms(docs_vec)

model = btm.BTM(X, vocabulary, seed=12321, T=10, M=20, alpha=50/8, beta=0.01)
model.fit(biterms, iterations=20)
p_zd = model.transform(docs_vec)

100%|██████████| 177692/177692 [00:02<00:00, 86223.60it/s]


In [53]:
perplexity = btm.perplexity(model.matrix_topics_words_, p_zd, X, 10)
coherence = btm.coherence(model.matrix_topics_words_, X, M=20)

In [54]:
btm.get_docs_top_topic(tweet_text_btm, model.matrix_docs_topics_)

,documents,label
0,"wirtschaftsjounalist:innen schreiben &amp; ""di...",1
1,https://t.co/pfnroveoas,0
2,"thread about the cruel german the debate, whi...",7
3,"liebe forsa - schön, per umfrage interessiert....",3
4,wichtiger thread arbeitsverträge begünstigt w...,9
...,...,...
177687,sprachlos &amp; stinksauer: schreibe seit 1 ja...,1
177688,deutscher hochschulverband (dhv): dhv legt eck...,6
177689,"schreibt kollegin noch, froh vertragsende arbe...",1
177690,"gerade erfahren, aufgrund 4-monatigen elternze...",0


In [55]:
import tmplot as tmp
tmp.report(model=model, docs=tweet_text_btm)

/usr/local/lib/python3.8/dist-packages/tmplot/_helpers.py:38: UserWarning: Please install "tomotopy" package to analyze its models.
Run `pip install tomotopy` in the console.
  warn(
/usr/local/lib/python3.8/dist-packages/tmplot/_helpers.py:38: UserWarning: Please install "tomotopy" package to analyze its models.
Run `pip install tomotopy` in the console.
  warn(
